In [16]:
import pandas as pd
import numpy as np
import json
import gensim
import pyLDAvis.gensim as gensimvis
import pyLDAvis
import os
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tokenizer.korean_tokenizer import tokenize_nori,tokenize_okt
from gensim.models import LdaModel
from os.path import join
from os import remove
from gensim.models import Word2Vec
from collections import Counter
from render import render
from tokenizer.english_tokenizer import tokenize_nltk
tqdm_notebook.pandas()


In [3]:
korean_df_folder = join('korean_crawling','articles')
for file in os.listdir(korean_df_folder):
    if file.endswith("merged.xlsx"):
        korean_df = pd.read_excel(join(korean_df_folder,file))
        korean_df.rename(columns={"urls" : "url","titles":"title","contents":"content"},inplace=True)
korean_df=korean_df.drop(['Unnamed: 0'],axis=1)
korean_df.head()

,url,title,content
0,https://www.donga.com/news/article/all/2017012...,야권서 흔들고 지지율은 안 오르고… 날카로워진 반기문,"[대선 정국]잇단 공격에 강공모드로 전환 MB, 반기문에 “파이팅” 반기문 전 유..."
1,https://www.donga.com/news/article/all/2017090...,"오준호 교수, ‘美 99년 만의 개기일식’ 동영상 공개",미국 전역에서 관측돼 세계인의 관심을 모았던 지난달 22일(현지 시간)의 개기일식...
2,https://www.donga.com/news/article/all/2017122...,[부고]권정현 별세·승단 승태 부친상·조성진 유준호 장인상 外,◇권정현 전 농수산부 양정국장 별세·...
3,https://www.donga.com/news/article/all/2017122...,"[Bio 의약]유럽 4종, 美1종, 韓4종… 바이오시밀러 제품 판매허가 획득",삼성바이오에피스\r 올해 미국-유럽서 잇단 판매 승인\r 자가면역질환 치료제뿐 아...
4,https://www.donga.com/news/article/all/2015120...,[부고]김상준 별세·예경 윤경 세진 부친상 外,◇김상준 전 ㈜풍산 대표이사 별세·예...


In [4]:
english_df = pd.DataFrame()
for file in os.listdir(join('english_crawling','result')):
    if file.endswith('.csv'):
        temp_df = pd.read_csv(join('english_crawling','result',file))
        english_df = pd.concat([english_df,temp_df])
english_df.rename(columns={"urls" : "url","titles":"title","contents":"content"},inplace=True)
english_df.head()

,content,title,url
0,Studies have already shown that by irradiating...,Wearable LED device could regrow hair,https://newatlas.com/led-hair-array/56306/
1,"January 25, 2005 A humanoid robot developed b...",Korea shows its robotics prowess with a rival ...,https://newatlas.com/korea-shows-its-robotics-...
2,"Make no mistake, we are in the middle of a pro...",From weapons to works of art: The year in arti...,https://newatlas.com/year-highlights-artificia...
3,A lot has been made of the dangers of drones m...,Flying sheepdogs: Herding birds away from plan...,https://newatlas.com/flying-sheepdogs-drones-p...
4,"If you build a soft, conformable robot body bu...","Thin-strip ""muscles"" may find use in soft n' s...",https://newatlas.com/artificial-muscles-soft-r...


In [5]:
journal_df = pd.DataFrame()
for folder in os.listdir(join('journal_crawling')):
    for file in os.listdir(join('journal_crawling',folder)):
        if file.endswith('.csv'):
            temp_df = pd.read_csv(join('journal_crawling',folder,file))
            temp_df.rename(columns={"urls" : "url","titles":"title","contents":"content"},inplace=True)
            journal_df = pd.concat([journal_df,temp_df])
journal_df=journal_df.drop(['Unnamed: 0'],axis=1)
journal_df.head()

,content,title,url
0,“Good ideas come to me when I am very comforta...,Won Do Heo,https://www.nature.com/articles/s41592-019-0626-1
1,The water/metal interface often governs import...,"Structure, Dynamics, and Wettability of Water ...",https://www.nature.com/articles/s41598-019-513...
2,In using nanostructures to design solar therma...,Design of a Broadband Solar Thermal Absorber U...,https://www.nature.com/articles/s41598-019-514...
3,Although glucose-sensing neurons were identifi...,A glucose-sensing neuron pair regulates insuli...,https://www.nature.com/articles/s41586-019-1675-4
4,"In tomographic reconstruction, the image quali...",Deep learning-based optical field screening fo...,https://www.nature.com/articles/s41598-019-513...


In [6]:
def identity_tokenizer(text):
    """
    sklearn의 Tfidfvectorizer를 이용함에 있어서 우리의 nori 토크나이저가 토큰화 한 결과를 사용하기 위한 method이다.

    :param text: 토큰화 하고자 하는 문장
    :return: 토큰이 담긴 리스트가 반환된다.
    """
    list1 = text.split('00')
    return list1

In [7]:
def TfidfWord(df):
    #TF-IDF를 기반으로 카이스트 검색시 가장 의미가 높은 단어로 예상되는 단어 100가지를 추출함.

    vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,
                                    max_features=100,
                                    max_df=0.5,
                                    ngram_range=(1, 1))

    # content를 기반으로 TF-IDF를 돌리는 함수
    words_list_content = []
    token_concat = df['content_token'].map(lambda x: "00".join(x))
    tfidf_matrix = vectorizer.fit_transform(token_concat)
    tfidf_wordslist = vectorizer.get_feature_names()
    vocab = dict()
    for idx, word in enumerate(tfidf_wordslist):
        vocab[word] = tfidf_matrix.getcol(idx).sum()
    words_list_content = sorted(vocab.items(), key=lambda x: x[1], reverse=True)


    # title을 기반으로 TF-IDF를 돌리는 함수
    words_list_title = []
    token_concat = df['title_token'].map(lambda x: "00".join(x))
    tfidf_matrix = vectorizer.fit_transform(token_concat)
    tfidf_wordslist = vectorizer.get_feature_names()
    vocab = dict()
    for idx, word in enumerate(tfidf_wordslist):
        vocab[word] = tfidf_matrix.getcol(idx).sum()
    words_list_title = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

    return words_list_content, words_list_title

In [8]:
def CountWord(df):
    #단어가 나온 빈도수를 기반으로 카이스트 검색시 가장 의미가 높은 단어로 예상되는 단어 100가지를 추출함.

    vectorizer = CountVectorizer(tokenizer=identity_tokenizer,
                                    max_features=100,
                                    max_df=0.5,
                                    ngram_range=(1, 1))

    # content를 기반으로 TF-IDF를 돌리는 함수
    words_list_content = []
    token_concat = df['content_token'].map(lambda x: "00".join(x))
    tfidf_matrix = vectorizer.fit_transform(token_concat)
    tfidf_wordslist = vectorizer.get_feature_names()
    vocab = dict()
    for idx, word in enumerate(tfidf_wordslist):
        vocab[word] = tfidf_matrix.getcol(idx).sum()
    words_list_content = sorted(vocab.items(), key=lambda x: x[1], reverse=True)


    # title을 기반으로 를 돌리는 함수
    words_list_title = []
    token_concat = df['title_token'].map(lambda x: "00".join(x))
    tfidf_matrix = vectorizer.fit_transform(token_concat)
    tfidf_wordslist = vectorizer.get_feature_names()
    vocab = dict()
    for idx, word in enumerate(tfidf_wordslist):
        vocab[word] = tfidf_matrix.getcol(idx).sum()
    words_list_title = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

    return words_list_content, words_list_title

In [9]:

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length


def topic_modeling(corpus_path,html_path):
    documents = Documents(corpus_path)
    dictionary = gensim.corpora.Dictionary(documents)
    min_count = 5
    word_counter = Counter((word for words in documents for word in words))
    removal_word_idxs = {
        dictionary.token2id[word] for word, count in word_counter.items()
        if count < min_count
    }

    dictionary.filter_tokens(removal_word_idxs)
    dictionary.compactify()
    corpus = Corpus(corpus_path, dictionary)
    lda_model = LdaModel(corpus, id2word=dictionary, num_topics=50)
    prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(prepared_data, html_path)


In [10]:
def find_cooccur(tokens,target,window,num):
    cooccur_dict = dict()
    for token in tokens:
        indices = [i for i, x in enumerate(token) if x.lower() == target.lower()]
        if len(indices)!=0:
            for indice in indices:
                for i in np.arange(indice-window,indice+window):
                    if i>=0 and i<=len(token)-1:
                        if token[i] in cooccur_dict.keys() :
                            cooccur_dict[token[i]] += 1
                        else :
                            cooccur_dict[token[i]] = 1 
    try :
        del cooccur_dict[target]
    except KeyError:
        try:
            del cooccur_dict[target.upper()]
        except KeyError:
            print(target)
    return sorted(cooccur_dict.items(),key=lambda item:item[1],reverse=True)[:num]

        

In [11]:
def stopwords_remove(dict):
    stopwords  = ['kaist','교수']
    for stopword in stopwords:
        try:
            del dict[stopword]
        except KeyError:
            continue
    return dict 

In [12]:
def word_tuple2dict(tups):
    target_dict =dict((x,int(y)) for x,y in tups)
    for key in list(target_dict.keys()):
        if len(key) ==1 :
           del target_dict[key]
    return target_dict

In [13]:
def dftotext(df,path):
    textlist = df['content_token'].tolist()
    with open(path,'w',encoding='utf-8-sig') as f:
        for text in textlist:
                f.write(' '.join(text)+'\n')

In [15]:
def visualize_korean(df,root_path,topic_num=10,keyword_num=50,topn=10):
    os.makedirs(root_path,exist_ok = True)
    print('Tokenize Start')
    #df =tokenize_nori(df)
    df = tokenize_okt(df)
    print('Tokenize End')
    detoken_path = join(root_path,'detokenize_text.txt')
    dftotext(df,detoken_path)
    print('detokenize end')
    topic_modeling(detoken_path, join(root_path,'topic_model.html'))
    print('topic model made')
    remove(detoken_path)

    #keyword extract part 
    TF_content_word, TF_title_word= TfidfWord(df)
    Count_content_word, Count_title_word= CountWord(df)
    # TF_content_word_dict=word_tuple2dict(TF_content_word[:keyword_num])
    # TF_content_word_dict = stopwords_remove(TF_content_word_dict)
    # TF_title_word_dict=word_tuple2dict(TF_title_word[:keyword_num])
    # TF_title_word_dict = stopwords_remove(TF_title_word_dict)
    Count_content_word_dict = word_tuple2dict(Count_content_word[:keyword_num])
    Count_content_word_dict = stopwords_remove(Count_content_word_dict)
    Count_title_word_dict = word_tuple2dict(Count_title_word[:keyword_num])
    Count_title_word_dict = stopwords_remove(Count_title_word_dict)


    dict_list = [Count_content_word_dict,Count_title_word_dict]
    titles_token=df['title_token'].tolist()
    contents_token=df['content_token'].tolist()
    tokens = titles_token + contents_token

    #window내에서 함께 나타난 다른 토큰들의 내림차순 및 유사어로 추측되는 것을 내림차순 정렬 그리고 저장
    model = Word2Vec(sentences=tokens,size=300,window=5,min_count=3,workers=4, sg=0)
    print('word2vec end')

    for j,method_dict in enumerate(dict_list):
        keyword_list = []
        for keyword in method_dict.keys():
            subkeyword_dict = dict()
            subkeyword_dict['keyword'] = keyword 
            subkeyword_dict['score'] = method_dict[keyword]

            tups = find_cooccur(tokens,keyword,4,topn)
            cooccur_list =[]
            for i,tup in enumerate(tups): 
                cooccur_list.append({'index':i+1, 'subkeyword':tup[0],'cooccur_num':tup[1]})
            subkeyword_dict['cooccur'] = cooccur_list
            try:
                tups = model.wv.similar_by_word(keyword,topn=topn)
            except :
                try:
                    tups = model.wv.similar_by_word(keyword.upper(),topn=topn)
                except : 
                    continue
            similar_list =[]
            for i,tup in enumerate(tups): 
                similar_list.append({'index':i+1, 'subkeyword':tup[0],'cooccur_num':tup[1]})
            subkeyword_dict['similar'] = similar_list

            keyword_list.append(subkeyword_dict)


        if j==0:
            with open(join(root_path,'content_wordcloud'+'.html'), 'w', encoding='UTF-8-sig') as file:
                file.write(render(json.dumps(keyword_list, ensure_ascii=False)))
        else:
            with open(join(root_path,'title_worldcloud'+'.html'), 'w', encoding='UTF-8-sig') as file:
                file.write(render(json.dumps(keyword_list, ensure_ascii=False)))
        print('made visualize file')
        
visualize_korean(korean_df,'./result/korean_visualize')
    

Tokenize Start



Tokenize End
detokenize end
topic model made
word2vec end
made visualize file

made visualize file


In [14]:
def visualize_english(df,root_path,topic_num=10,keyword_num=50,topn=10):
    os.makedirs(root_path,exist_ok = True)
    print('Tokenize Start')
    df = tokenize_nltk(df)
    print('tokenize End')
    detoken_path = join(root_path,'detokenize_text.txt')
    dftotext(df,detoken_path)
    print('detokenize end')
    topic_modeling(detoken_path, join(root_path,'topic_model.html'))
    print('topic model made')
    remove(detoken_path)

    #keyword extract part 
    TF_content_word, TF_title_word= TfidfWord(df)
    Count_content_word, Count_title_word= CountWord(df)
    # TF_content_word_dict=word_tuple2dict(TF_content_word[:keyword_num])
    # TF_content_word_dict = stopwords_remove(TF_content_word_dict)
    # TF_title_word_dict=word_tuple2dict(TF_title_word[:keyword_num])
    # TF_title_word_dict = stopwords_remove(TF_title_word_dict)
    Count_content_word_dict=word_tuple2dict(Count_content_word[:keyword_num])
    Count_content_word_dict = stopwords_remove(Count_content_word_dict)
    Count_title_word_dict=word_tuple2dict(Count_title_word[:keyword_num])
    Count_title_word_dict = stopwords_remove(Count_title_word_dict)


    dict_list = [Count_content_word_dict,Count_title_word_dict]
    titles_token=df['title_token'].tolist()
    contents_token=df['content_token'].tolist()
    tokens = titles_token + contents_token

    #window내에서 함께 나타난 다른 토큰들의 내림차순 및 유사어로 추측되는 것을 내림차순 정렬 그리고 저장
    model = Word2Vec(sentences=tokens,size=300,window=5,min_count=3,workers=4, sg=0)
    print('word2vec end')

    for j,method_dict in enumerate(dict_list):
        keyword_list = []
        for keyword in method_dict.keys():
            subkeyword_dict = dict()
            subkeyword_dict['keyword'] = keyword 
            subkeyword_dict['score'] = method_dict[keyword]

            tups = find_cooccur(tokens,keyword,4,topn)
            cooccur_list =[]
            for i,tup in enumerate(tups): 
                cooccur_list.append({'index':i+1, 'subkeyword':tup[0],'cooccur_num':tup[1]})
            subkeyword_dict['cooccur'] = cooccur_list
            try:
                tups = model.wv.similar_by_word(keyword,topn=topn)
            except :
                try:
                    tups = model.wv.similar_by_word(keyword.upper(),topn=topn)
                except : 
                    continue
            similar_list =[]
            for i,tup in enumerate(tups): 
                similar_list.append({'index':i+1, 'subkeyword':tup[0],'cooccur_num':tup[1]})
            subkeyword_dict['similar'] = similar_list

            keyword_list.append(subkeyword_dict)


        if j==0:
            with open(join(root_path,'content_wordcloud'+'.html'), 'w', encoding='UTF-8-sig') as file:
                file.write(render(json.dumps(keyword_list, ensure_ascii=False)))
        else:
            with open(join(root_path,'title_worldcloud'+'.html'), 'w', encoding='UTF-8-sig') as file:
                file.write(render(json.dumps(keyword_list, ensure_ascii=False)))
        print('made visualize file')
        
visualize_english(english_df,'./result/english_visualize')
visualize_english(journal_df,'./result/journal_visualize')

Tokenize Start



tokenize End
detokenize end
topic model made
word2vec end
made visualize file
made visualize file
